## outline

1. introduction  / problem description
1. basics from graph theory
1. mathematical formulation
1. pyomo implementation
1. example
1. summary or outlook for other solution methods

reference: http://cgm.cs.mcgill.ca/~avis/courses/251/2012/ktlaerid/LP-bliss.pdf